In [1]:
import pandas as pd
import requests
from tqdm import tqdm  # <-- Progress bar
import time

# Load input CSV
input_file = 'coor_600.csv'
df = pd.read_csv(input_file)

# Base API config
base_url = "https://archive-api.open-meteo.com/v1/archive"
start_date = "2011-01-01"
end_date = "2011-12-31"

# Store results
combined_data = []

def fetch_weather_data(row):
    lat = row['latitude']
    lon = row['longitude']
    state = row['State_Name']
    district = row['District_Name']

    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": start_date,
        "end_date": end_date,
        "daily": "precipitation_sum,relative_humidity_2m_mean,temperature_2m_mean,wind_speed_10m_mean"
    }

    try:
        response = requests.get(base_url, params=params, timeout=30)
        response.raise_for_status()
        data = response.json()

        results = []
        if 'daily' in data:
            for i in range(len(data['daily']['time'])):
                results.append({
                    "state": state,
                    "district": district,
                    "latitude": lat,
                    "longitude": lon,
                    "date": data['daily']['time'][i],
                    "precipitation_sum": data['daily']['precipitation_sum'][i],
                    "relative_humidity_2m_mean": data['daily']['relative_humidity_2m_mean'][i],
                    "temperature_2m_mean": data['daily']['temperature_2m_mean'][i],
                    "wind_speed_10m_mean": data['daily']['wind_speed_10m_mean'][i]
                })
        return results

    except Exception as e:
        print(f"[x] Failed for {district}, {state}: {e}")
        return []

# Sequential execution with progress bar
for _, row in tqdm(df.iterrows(), total=len(df), desc="Fetching weather data"):
    combined_data.extend(fetch_weather_data(row))
    time.sleep(1)  # Delay to avoid hitting rate limits

# Save to CSV
output_file = "historical_weather_600_data20111.csv"
pd.DataFrame(combined_data).to_csv(output_file, index=False)
print(f"\n✅ All data saved to '{output_file}'")

Fetching weather data: 100%|█████████████████████████████████████████████████████████| 321/321 [10:28<00:00,  1.96s/it]



✅ All data saved to 'historical_weather_600_data20111.csv'
